In [ ]:
import unittest
from unittest.mock import patch, MagicMock
import sys
import tkinter as tk
import pandas as pd
from datetime import datetime
from NewsAggregator import NewsAPI_KeyList, NewsScraper  # Assuming your main file is named NewsAggregator.py

class TestKeyList(unittest.TestCase):
    def setUp(self):
        self.keylist = NewsAPI_KeyList()
        self.test_key = '513d1d7c31e34a49aeedadeccd4f58e5'
        self.invalid_key = 'shortkey'

    def test_add_valid_key(self):
        self.keylist.add_key(self.test_key)
        self.assertIn(self.test_key, self.keylist._keys)

    def test_add_invalid_key(self):
        with self.assertRaises(ValueError):
            self.keylist.add_key(self.invalid_key)

    @patch('requests.get')
    def test_get_key_with_valid_api_check(self, mock_get):
        # Setup mock response
        mock_response = MagicMock()
        mock_response.status_code = 200
        mock_response.json.return_value = {'status': 'ok'}
        mock_get.return_value = mock_response

        self.keylist.add_key(self.test_key)
        self.keylist.set_url('https://newsapi.org/v2/top-headlines/sources')
        result = self.keylist.get_key()
        self.assertEqual(result, self.test_key)

class TestNewsScraper(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        # Create a root window once for all tests
        cls.root = tk.Tk()
        cls.root.withdraw()  # Hide the window during tests

    @classmethod
    def tearDownClass(cls):
        cls.root.destroy()

    def setUp(self):
        self.scraper = NewsScraper(self.root)

    def test_initialization(self):
        self.assertIsInstance(self.scraper, NewsScraper)
        self.assertIsNotNone(self.scraper.api_key)
        self.assertIsInstance(self.scraper.df_articles, pd.DataFrame)

    def test_get_sources(self):
        sources = self.scraper.get_sources()
        self.assertIsInstance(sources, dict)
        if sources:  # Only check contents if we got data
            for key, value in sources.items():
                self.assertIsInstance(key, str)
                self.assertIsInstance(value, str)

    def test_get_countries(self):
        countries = self.scraper.get_countries()
        self.assertIsInstance(countries, dict)
        if countries:
            for key, value in countries.items():
                self.assertIsInstance(key, str)
                self.assertIsInstance(value, str)

    def test_get_categories(self):
        categories = self.scraper.get_categories()
        self.assertIsInstance(categories, list)
        if categories:
            for category in categories:
                self.assertIsInstance(category, str)

    @patch('requests.get')
    def test_get_api_news_with_source(self, mock_get):
        # Setup mock response
        mock_response = MagicMock()
        mock_response.status_code = 200
        mock_response.json.return_value = {
            'status': 'ok',
            'articles': [{
                'title': 'Test Article',
                'source': {'name': 'Test Source'},
                'publishedAt': '2023-01-01T00:00:00Z',
                'author': 'Test Author',
                'description': 'Test Description',
                'url': 'http://test.com'
            }]
        }
        mock_get.return_value = mock_response

        # Simulate source selection
        self.scraper.answer.set("0")
        self.scraper.source_listbox.insert(0, "Test Source")
        self.scraper.source_listbox.selection_set(0)

        self.scraper.get_api_news()
        self.assertFalse(self.scraper.df_articles.empty)
        self.assertEqual(self.scraper.df_articles.iloc[0]['Title'], 'Test Article')

    @patch('requests.get')
    def test_get_api_news_with_category_country(self, mock_get):
        # Setup mock response
        mock_response = MagicMock()
        mock_response.status_code = 200
        mock_response.json.return_value = {
            'status': 'ok',
            'articles': [{
                'title': 'Test Article',
                'source': {'name': 'Test Source'},
                'publishedAt': '2023-01-01T00:00:00Z',
                'author': 'Test Author',
                'description': 'Test Description',
                'url': 'http://test.com'
            }]
        }
        mock_get.return_value = mock_response

        # Simulate category/country selection
        self.scraper.answer.set("1")
        self.scraper.category_listbox.insert(0, "business")
        self.scraper.country_listbox.insert(0, "Australia")
        self.scraper.category_listbox.selection_set(0)
        self.scraper.country_listbox.selection_set(0)

        self.scraper.get_api_news()
        self.assertFalse(self.scraper.df_articles.empty)
        self.assertEqual(self.scraper.df_articles.iloc[0]['Title'], 'Test Article')

    @patch('requests.get')
    def test_get_ABC_news(self, mock_get):
        # Setup mock responses
        mock_main_page = MagicMock()
        mock_main_page.text = """
        <script type="application/ld+json">
        {
            "@context": "https://schema.org",
            "@type": "ItemList",
            "itemListElement": [
                {"@type": "ListItem", "url": "http://abc.net.au/news/test1"},
                {"@type": "ListItem", "url": "http://abc.net.au/news/test2"}
            ]
        }
        </script>
        """
        
        mock_article_page = MagicMock()
        mock_article_page.text = """
        <script type="application/ld+json">
        {
            "@type": "NewsArticle",
            "headline": "ABC Test Article",
            "datePublished": "2023-01-01T00:00:00Z",
            "publisher": {"name": "ABC News"},
            "author": [{"name": "ABC Author"}],
            "description": "ABC Test Description"
        }
        </script>
        """
        
        mock_get.side_effect = [mock_main_page, mock_article_page, mock_article_page]

        self.scraper.get_ABC_news()
        self.assertFalse(self.scraper.df_articles.empty)
        self.assertEqual(self.scraper.df_articles.iloc[0]['Title'], 'ABC Test Article')

    def test_fill_textbox(self):
        # Create test data
        test_data = {
            'Title': ['Test Title'],
            'Source': ['Test Source'],
            'PublishedDate': ['01 Jan 2023 12:00 PM'],
            'Author': ['Test Author'],
            'Description': ['Test Description'],
            'ExtractionMethod': ['Test'],
            'URL': ['http://test.com']
        }
        self.scraper.df_articles = pd.DataFrame(test_data)
        
        self.scraper.fill_textbox()
        textbox_content = self.scraper.output_textbox.get("1.0", tk.END)
        self.assertIn('Test Title', textbox_content)
        self.assertIn('Test Source', textbox_content)

    def test_update_plots(self):
        # Create test data with titles
        test_data = {
            'Title': ['Python is great', 'Python testing is important', 'Great Python tools'],
            'Source': ['Test1', 'Test2', 'Test3']
        }
        self.scraper.df_articles = pd.DataFrame(test_data)
        
        self.scraper.update_plots()
        # Verify the plot was updated (we can't easily test the visual output, but we can check no errors occurred)
        self.assertTrue(True)  # Placeholder assertion

    def test_update_plots_empty_data(self):
        # Test with empty DataFrame
        self.scraper.df_articles = pd.DataFrame()
        self.scraper.update_plots()
        # Just verify no exceptions were raised
        self.assertTrue(True)

if __name__ == '__main__':
    unittest.main(argv=[' '], exit=False)

    

...

In [ ]:
import unittest
from unittest.mock import patch, MagicMock
import tkinter as tk
import pandas as pd
from datetime import datetime

# Replace with your actual import, make sure NewsAggregator.py is in the same directory or in Python path
from NewsAggregator import NewsAPI_KeyList, NewsScraper



In [ ]:
class TestKeyList(unittest.TestCase):
    def setUp(self):
        self.keylist = NewsAPI_KeyList()
        self.test_key = '513d1d7c31e34a49aeedadeccd4f58e5'
        self.invalid_key = 'shortkey'

    def test_add_valid_key(self):
        self.keylist.add_key(self.test_key)
        self.assertIn(self.test_key, self.keylist._keys)

    def test_add_invalid_key(self):
        with self.assertRaises(ValueError):
            self.keylist.add_key(self.invalid_key)


    @patch('requests.get')
    def test_get_key_with_valid_api_check(self, mock_get):
        mock_response = MagicMock()
        mock_response.status_code = 200
        mock_response.json.return_value = {'status': 'ok'}
        mock_get.return_value = mock_response

        self.keylist.add_key(self.test_key)
        self.keylist.set_url('https://newsapi.org/v2/top-headlines/sources')
        result = self.keylist.get_key()
        self.assertEqual(result, self.test_key)


In [ ]:
class TestNewsScraper(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        cls.root = tk.Tk()
        cls.root.withdraw()

    @classmethod
    def tearDownClass(cls):
        cls.root.destroy()

    def setUp(self):
        self.scraper = NewsScraper(self.root)


In [ ]:
    def test_initialization(self):
        self.assertIsInstance(self.scraper, NewsScraper)
        self.assertIsNotNone(self.scraper.api_key)
        self.assertIsInstance(self.scraper.df_articles, pd.DataFrame)

    def test_get_sources(self):
        sources = self.scraper.get_sources()
        self.assertIsInstance(sources, dict)
        if sources:
            for key, value in sources.items():
                self.assertIsInstance(key, str)
                self.assertIsInstance(value, str)

    def test_get_countries(self):
        countries = self.scraper.get_countries()
        self.assertIsInstance(countries, dict)
        if countries:
            for key, value in countries.items():
                self.assertIsInstance(key, str)
                self.assertIsInstance(value, str)

    def test_get_categories(self):
        categories = self.scraper.get_categories()
        self.assertIsInstance(categories, list)
        if categories:
            for category in categories:
                self.assertIsInstance(category, str)


In [ ]:
    @patch('requests.get')
    def test_get_api_news_with_source(self, mock_get):
        mock_response = MagicMock()
        mock_response.status_code = 200
        mock_response.json.return_value = {
            'status': 'ok',
            'articles': [{
                'title': 'Test Article',
                'source': {'name': 'Test Source'},
                'publishedAt': '2023-01-01T00:00:00Z',
                'author': 'Test Author',
                'description': 'Test Description',
                'url': 'http://test.com'
            }]
        }
        mock_get.return_value = mock_response

        self.scraper.answer.set("0")
        self.scraper.source_listbox.insert(0, "Test Source")
        self.scraper.source_listbox.selection_set(0)

        self.scraper.get_api_news()
        self.assertFalse(self.scraper.df_articles.empty)
        self.assertEqual(self.scraper.df_articles.iloc[0]['Title'], 'Test Article')


In [ ]:
    @patch('requests.get')
    def test_get_api_news_with_category_country(self, mock_get):
        mock_response = MagicMock()
        mock_response.status_code = 200
        mock_response.json.return_value = {
            'status': 'ok',
            'articles': [{
                'title': 'Test Article',
                'source': {'name': 'Test Source'},
                'publishedAt': '2023-01-01T00:00:00Z',
                'author': 'Test Author',
                'description': 'Test Description',
                'url': 'http://test.com'
            }]
        }
        mock_get.return_value = mock_response

        self.scraper.answer.set("1")
        self.scraper.category_listbox.insert(0, "business")
        self.scraper.country_listbox.insert(0, "Australia")
        self.scraper.category_listbox.selection_set(0)
        self.scraper.country_listbox.selection_set(0)

        self.scraper.get_api_news()
        self.assertFalse(self.scraper.df_articles.empty)
        self.assertEqual(self.scraper.df_articles.iloc[0]['Title'], 'Test Article')


In [ ]:
    @patch('requests.get')
    def test_get_ABC_news(self, mock_get):
        mock_main_page = MagicMock()
        mock_main_page.text = """
        <script type="application/ld+json">
        {
            "@context": "https://schema.org",
            "@type": "ItemList",
            "itemListElement": [
                {"@type": "ListItem", "url": "http://abc.net.au/news/test1"},
                {"@type": "ListItem", "url": "http://abc.net.au/news/test2"}
            ]
        }
        </script>
        """
        
        mock_article_page = MagicMock()
        mock_article_page.text = """
        <script type="application/ld+json">
        {
            "@type": "NewsArticle",
            "headline": "ABC Test Article",
            "datePublished": "2023-01-01T00:00:00Z",
            "publisher": {"name": "ABC News"},
            "author": [{"name": "ABC Author"}],
            "description": "ABC Test Description"
        }
        </script>
        """
        
        mock_get.side_effect = [mock_main_page, mock_article_page, mock_article_page]

        self.scraper.get_ABC_news()
        self.assertFalse(self.scraper.df_articles.empty)
        self.assertEqual(self.scraper.df_articles.iloc[0]['Title'], 'ABC Test Article')


In [ ]:
    def test_fill_textbox(self):
        test_data = {
            'Title': ['Test Title'],
            'Source': ['Test Source'],
            'PublishedDate': ['01 Jan 2023 12:00 PM'],
            'Author': ['Test Author'],
            'Description': ['Test Description'],
            'ExtractionMethod': ['Test'],
            'URL': ['http://test.com']
        }
        self.scraper.df_articles = pd.DataFrame(test_data)
        
        self.scraper.fill_textbox()
        textbox_content = self.scraper.output_textbox.get("1.0", tk.END)
        self.assertIn('Test Title', textbox_content)
        self.assertIn('Test Source', textbox_content)

    def test_update_plots(self):
        test_data = {
            'Title': ['Python is great', 'Python testing is important', 'Great Python tools'],
            'Source': ['Test1', 'Test2', 'Test3']
        }
        self.scraper.df_articles = pd.DataFrame(test_data)
        self.scraper.update_plots()
        self.assertTrue(True)  # Just checking for no exceptions

    def test_update_plots_empty_data(self):
        self.scraper.df_articles = pd.DataFrame()
        self.scraper.update_plots()
        self.assertTrue(True)


In [ ]:
unittest.main(argv=[''], exit=False)